In [1]:
%AddDeps org.apache.bahir spark-sql-streaming-mqtt_2.11 2.1.2 -transitive

import org.apache.spark.sql.SparkSession
import java.sql.Timestamp

Marking org.apache.bahir:spark-sql-streaming-mqtt_2.11:2.1.2 for download
Preparing to fetch from:
-> file:/gpfs/fs01/user/s1a7-203321b462d21d-1bc0acfaf2fe/notebook/tmp/toree-tmp-dir8751921129012930481/toree_add_deps/
-> https://repo1.maven.org/maven2
-> New file at /gpfs/fs01/user/s1a7-203321b462d21d-1bc0acfaf2fe/notebook/tmp/toree-tmp-dir8751921129012930481/toree_add_deps/https/repo1.maven.org/maven2/org/eclipse/paho/org.eclipse.paho.client.mqttv3/1.1.0/org.eclipse.paho.client.mqttv3-1.1.0.jar
-> New file at /gpfs/fs01/user/s1a7-203321b462d21d-1bc0acfaf2fe/notebook/tmp/toree-tmp-dir8751921129012930481/toree_add_deps/https/repo1.maven.org/maven2/org/apache/bahir/spark-sql-streaming-mqtt_2.11/2.1.2/spark-sql-streaming-mqtt_2.11-2.1.2.jar
-> New file at /gpfs/fs01/user/s1a7-203321b462d21d-1bc0acfaf2fe/notebook/tmp/toree-tmp-dir8751921129012930481/toree_add_deps/https/repo1.maven.org/maven2/org/apache/spark/spark-tags_2.11/2.1.2/spark-tags_2.11-2.1.2.jar
-> New file at /gpfs/fs01/user/s1

Waiting for a Spark session to start...

In [2]:
val spark = SparkSession.builder().getOrCreate()

In [3]:
case class WaterConsumptionClass(DayOfWeek: Double, Consumption: Double)
import spark.implicits._
val df1 = Seq(
(1.0,1000.0),
(2.0,1050.0),
(3.0,850.0),
(4.0,950.0),
(5.0,1100.0),
(6.0,1300.0),
(7.0,1400.0),
(1.0,850.0),
(2.0,950.0),
(3.0,1100.0),
(4.0,1000.0),
(5.0,1050.0),
(6.0,850.0),
(7.0,950.0),
(1.0,950.0),
(2.0,1200.0),
(3.0,850.0),
(4.0,970.0),
(5.0,1010.0),
(6.0,1210.0),
(7.0,1310.0),
(1.0,1410.0),
(2.0,1200.0),
(3.0,890.0),
(4.0,930.0),
(5.0,1200.0),
(6.0,1105.0),
(7.0,1202.0)
).toDF("DayOfWeek","WaterConsumption")
df1.show(5)
df1.printSchema()
df1.schema

+---------+----------------+
|DayOfWeek|WaterConsumption|
+---------+----------------+
|      1.0|          1000.0|
|      2.0|          1050.0|
|      3.0|           850.0|
|      4.0|           950.0|
|      5.0|          1100.0|
+---------+----------------+
only showing top 5 rows

root
 |-- DayOfWeek: double (nullable = false)
 |-- WaterConsumption: double (nullable = false)



StructType(StructField(DayOfWeek,DoubleType,false), StructField(WaterConsumption,DoubleType,false))

In [4]:
val df2 = df1.select(df1("WaterConsumption").as("label"), df1("DayOfWeek").as("features1"))
df2.show()

+------+---------+
| label|features1|
+------+---------+
|1000.0|      1.0|
|1050.0|      2.0|
| 850.0|      3.0|
| 950.0|      4.0|
|1100.0|      5.0|
|1300.0|      6.0|
|1400.0|      7.0|
| 850.0|      1.0|
| 950.0|      2.0|
|1100.0|      3.0|
|1000.0|      4.0|
|1050.0|      5.0|
| 850.0|      6.0|
| 950.0|      7.0|
| 950.0|      1.0|
|1200.0|      2.0|
| 850.0|      3.0|
| 970.0|      4.0|
|1010.0|      5.0|
|1210.0|      6.0|
+------+---------+
only showing top 20 rows



In [5]:
import org.apache.spark.ml.regression._
import org.apache.spark.ml.feature.VectorAssembler
val assembler = new VectorAssembler().setInputCols(Array("features1")).setOutputCol("features")
val df3 = assembler.transform(df2).select($"label",$"features")
df3.show(5)

+------+--------+
| label|features|
+------+--------+
|1000.0|   [1.0]|
|1050.0|   [2.0]|
| 850.0|   [3.0]|
| 950.0|   [4.0]|
|1100.0|   [5.0]|
+------+--------+
only showing top 5 rows



In [6]:
val lr = new LinearRegression()
val lrModel = lr.fit(df3)
println(s"Slope/Coefficients: ${lrModel.coefficients} Intercept: ${lrModel.intercept}")

val trainingSummary = lrModel.summary

println(s"numIterations: ${trainingSummary.totalIterations}")
println(s"objectiveHistory: ${trainingSummary.objectiveHistory.toList}")

trainingSummary.residuals.show()

println(s"RMSE: ${trainingSummary.rootMeanSquaredError}")
println(s"MSE: ${trainingSummary.meanSquaredError}")
println(s"r2: ${trainingSummary.r2}")

[Stage 0:>                                                          (0 + 2) / 2]Slope/Coefficients: [24.60714285714254] Intercept: 967.1785714285727
numIterations: 1
objectiveHistory: List(0.0)
+-------------------+
|          residuals|
+-------------------+
|  8.214285714284756|
|  33.60714285714221|
|-191.00000000000023|
|-115.60714285714289|
|  9.785714285714675|
|   185.178571428572|
|  260.5714285714296|
|-141.78571428571524|
| -66.39285714285779|
|  58.99999999999977|
| -65.60714285714289|
|-40.214285714285325|
|  -264.821428571428|
|-189.42857142857042|
|-41.785714285715244|
|  183.6071428571422|
|-191.00000000000023|
| -95.60714285714289|
| -80.21428571428532|
|  95.17857142857201|
+-------------------+
only showing top 20 rows

RMSE: 156.38681357193667
MSE: 24456.835459183672
r2: 0.09010962488901075


In [13]:
val df4 = lrModel.transform(df3)
df4.show(4)
import org.apache.spark.sql.Row
import org.apache.spark.ml.linalg.{Vector,Vectors}
df4.printSchema

val df5 = df4.map{case Row(d1:Double, v: org.apache.spark.ml.linalg.Vector, d2:Double) => (d1,v(0),d2)}.toDF("WaterConsumption","DayOfWeek","Prediction")
df5.show(4)

+------+--------+------------------+
| label|features|        prediction|
+------+--------+------------------+
|1000.0|   [1.0]| 991.7857142857152|
|1050.0|   [2.0]|1016.3928571428578|
| 850.0|   [3.0]|1041.0000000000002|
| 950.0|   [4.0]| 1065.607142857143|
+------+--------+------------------+
only showing top 4 rows

root
 |-- label: double (nullable = false)
 |-- features: vector (nullable = true)
 |-- prediction: double (nullable = true)

+----------------+---------+------------------+
|WaterConsumption|DayOfWeek|        Prediction|
+----------------+---------+------------------+
|          1000.0|      1.0| 991.7857142857152|
|          1050.0|      2.0|1016.3928571428578|
|           850.0|      3.0|1041.0000000000002|
|           950.0|      4.0| 1065.607142857143|
+----------------+---------+------------------+
only showing top 4 rows



In [8]:
%AddDeps org.vegas-viz vegas_2.11 0.3.11
%AddDeps org.vegas-viz vegas-spark_2.11 0.3.11 
%AddDeps org.typelevel cats-core_2.11 0.9.0
%AddDeps org.typelevel cats-kernel_2.11 0.9.0
%AddDeps org.typelevel cats-macros_2.11 0.9.0
%AddDeps io.circe circe-core_2.11 0.7.0
%AddDeps io.circe circe-generic_2.11 0.7.0
%AddDeps io.circe circe-jawn_2.11 0.7.0
%AddDeps io.circe circe-numbers_2.11 0.7.0
%AddDeps io.circe circe-parser_2.11 0.7.0
%AddDeps org.spire-math jawn-parser_2.11 0.10.4
%AddDeps org.typelevel machinist_2.11 0.6.1
%AddDeps org.typelevel macro-compat_2.11 1.1.1
%AddDeps com.github.julien-truffaut monocle-core_2.11 1.1.0
%AddDeps com.github.julien-truffaut monocle-macro_2.11 1.1.0
%AddDeps org.scalafx scalafx_2.11-8 0.92-R10
%AddDeps org.scalaz scalaz-core_2.11 7.1.1
%AddDeps com.chuusai shapeless_2.11 2.3.2
%AddDeps com.github.mpilquist simulacrum_2.11 0.10.0
%AddDeps org.webjars.bower vega-3 0.0-rc4
%AddDeps org.webjars.bower vega-lite 1.2.0
%AddDeps org.vegas-viz vegas_2.11 0.3.11
%AddDeps org.vegas-viz vegas-macros_2.11 0.3.11

Marking org.vegas-viz:vegas_2.11:0.3.11 for download
Preparing to fetch from:
-> file:/gpfs/fs01/user/s1a7-203321b462d21d-1bc0acfaf2fe/notebook/tmp/toree-tmp-dir8751921129012930481/toree_add_deps/
-> https://repo1.maven.org/maven2
-> New file at /gpfs/fs01/user/s1a7-203321b462d21d-1bc0acfaf2fe/notebook/tmp/toree-tmp-dir8751921129012930481/toree_add_deps/https/repo1.maven.org/maven2/org/vegas-viz/vegas_2.11/0.3.11/vegas_2.11-0.3.11.jar
Marking org.vegas-viz:vegas-spark_2.11:0.3.11 for download
Preparing to fetch from:
-> file:/gpfs/fs01/user/s1a7-203321b462d21d-1bc0acfaf2fe/notebook/tmp/toree-tmp-dir8751921129012930481/toree_add_deps/
-> https://repo1.maven.org/maven2
-> New file at /gpfs/fs01/user/s1a7-203321b462d21d-1bc0acfaf2fe/notebook/tmp/toree-tmp-dir8751921129012930481/toree_add_deps/https/repo1.maven.org/maven2/org/vegas-viz/vegas-spark_2.11/0.3.11/vegas-spark_2.11-0.3.11.jar
Marking org.typelevel:cats-core_2.11:0.9.0 for download
Preparing to fetch from:
-> file:/gpfs/fs01/user

Marking com.github.mpilquist:simulacrum_2.11:0.10.0 for download
Preparing to fetch from:
-> file:/gpfs/fs01/user/s1a7-203321b462d21d-1bc0acfaf2fe/notebook/tmp/toree-tmp-dir8751921129012930481/toree_add_deps/
-> https://repo1.maven.org/maven2
-> New file at /gpfs/fs01/user/s1a7-203321b462d21d-1bc0acfaf2fe/notebook/tmp/toree-tmp-dir8751921129012930481/toree_add_deps/https/repo1.maven.org/maven2/com/github/mpilquist/simulacrum_2.11/0.10.0/simulacrum_2.11-0.10.0.jar
Marking org.webjars.bower:vega-3:0.0-rc4 for download
Preparing to fetch from:
-> file:/gpfs/fs01/user/s1a7-203321b462d21d-1bc0acfaf2fe/notebook/tmp/toree-tmp-dir8751921129012930481/toree_add_deps/
-> https://repo1.maven.org/maven2
-> Failed to resolve org.webjars.bower:vega-3:0.0-rc4
    -> not found: /gpfs/fs01/user/s1a7-203321b462d21d-1bc0acfaf2fe/notebook/tmp/toree-tmp-dir8751921129012930481/toree_add_deps/cache/org.webjars.bower/vega-3/ivy-0.0-rc4.xml
    -> not found: https://repo1.maven.org/maven2/org/webjars/bower/vega

In [12]:
import vegas._
import vegas.data.External._
import vegas.sparkExt._
Vegas.layered("Linear Regression", width=500, height=500).
  withLayers(
    Layer().
        withDataFrame(df1).
        mark(Point).
        encodeX("DayOfWeek", Quant).
        encodeY("WaterConsumption", Quant),
    Layer().
      withDataFrame(df5).
      mark(Line).
      encodeX("DayOfWeek", Quant).
      encodeY("Prediction", Quant)
  ).
  show

<iframe id="frame-vegas-2781fe27-4c98-4a39-ae7f-27b049650d99" sandbox="allow-scripts allow-same-origin" style="border: none; width: 100%" srcdoc="<html>
 <head>
 <script src="https://cdn.jsdelivr.net/webjars/org.webjars.bower/d3/3.5.17/d3.min.js" charset="utf-8"></script>
<script src="https://cdn.jsdelivr.net/webjars/org.webjars.bower/vega/2.6.3/vega.min.js" charset="utf-8"></script>
<script src="https://cdn.jsdelivr.net/webjars/org.webjars.bower/vega-lite/1.2.0/vega-lite.min.js" charset="utf-8"></script>
<script src="https://vega.github.io/vega-editor/vendor/vega-embed.js" charset="utf-8"></script>
 </head>
 <body>
 <div id='vegas-2781fe27-4c98-4a39-ae7f-27b049650d99'></div>
 <script>
 var embedSpec = {
 mode: "vega-lite",
 spec: {
 "width" : 500.0,
 "height" : 500.0,
 "layers" : [
 {
 "mark" : "point",
 "encoding" : {
 "x" : {
 "field" : "DayOfWeek",
 "type" : "quantitative"
 },
 "y" : {
 "field" : "WaterConsumption",
 "type" : "quantitative"
 }
 },
 "data" : {
 "values" : [
 {
 "DayOfWeek" : 1.0,
 "WaterConsumption" : 1000.0
 },
 {
 "DayOfWeek" : 2.0,
 "WaterConsumption" : 1050.0
 },
 {
 "DayOfWeek" : 3.0,
 "WaterConsumption" : 850.0
 },
 {
 "DayOfWeek" : 4.0,
 "WaterConsumption" : 950.0
 },
 {
 "DayOfWeek" : 5.0,
 "WaterConsumption" : 1100.0
 },
 {
 "DayOfWeek" : 6.0,
 "WaterConsumption" : 1300.0
 },
 {
 "DayOfWeek" : 7.0,
 "WaterConsumption" : 1400.0
 },
 {
 "DayOfWeek" : 1.0,
 "WaterConsumption" : 850.0
 },
 {
 "DayOfWeek" : 2.0,
 "WaterConsumption" : 950.0
 },
 {
 "DayOfWeek" : 3.0,
 "WaterConsumption" : 1100.0
 },
 {
 "DayOfWeek" : 4.0,
 "WaterConsumption" : 1000.0
 },
 {
 "DayOfWeek" : 5.0,
 "WaterConsumption" : 1050.0
 },
 {
 "DayOfWeek" : 6.0,
 "WaterConsumption" : 850.0
 },
 {
 "DayOfWeek" : 7.0,
 "WaterConsumption" : 950.0
 },
 {
 "DayOfWeek" : 1.0,
 "WaterConsumption" : 950.0
 },
 {
 "DayOfWeek" : 2.0,
 "WaterConsumption" : 1200.0
 },
 {
 "DayOfWeek" : 3.0,
 "WaterConsumption" : 850.0
 },
 {
 "DayOfWeek" : 4.0,
 "WaterConsumption" : 970.0
 },
 {
 "DayOfWeek" : 5.0,
 "WaterConsumption" : 1010.0
 },
 {
 "DayOfWeek" : 6.0,
 "WaterConsumption" : 1210.0
 },
 {
 "DayOfWeek" : 7.0,
 "WaterConsumption" : 1310.0
 },
 {
 "DayOfWeek" : 1.0,
 "WaterConsumption" : 1410.0
 },
 {
 "DayOfWeek" : 2.0,
 "WaterConsumption" : 1200.0
 },
 {
 "DayOfWeek" : 3.0,
 "WaterConsumption" : 890.0
 },
 {
 "DayOfWeek" : 4.0,
 "WaterConsumption" : 930.0
 },
 {
 "DayOfWeek" : 5.0,
 "WaterConsumption" : 1200.0
 },
 {
 "DayOfWeek" : 6.0,
 "WaterConsumption" : 1105.0
 },
 {
 "DayOfWeek" : 7.0,
 "WaterConsumption" : 1202.0
 }
 ]
 }
 },
 {
 "mark" : "line",
 "encoding" : {
 "x" : {
 "field" : "DayOfWeek",
 "type" : "quantitative"
 },
 "y" : {
 "field" : "Prediction",
 "type" : "quantitative"
 }
 },
 "data" : {
 "values" : [
 {
 "WaterConsumption" : 1000.0,
 "DayOfWeek" : 1.0,
 "Prediction" : 991.7857142857152
 },
 {
 "WaterConsumption" : 1050.0,
 "DayOfWeek" : 2.0,
 "Prediction" : 1016.3928571428578
 },
 {
 "WaterConsumption" : 850.0,
 "DayOfWeek" : 3.0,
 "Prediction" : 1041.0000000000002
 },
 {
 "WaterConsumption" : 950.0,
 "DayOfWeek" : 4.0,
 "Prediction" : 1065.607142857143
 },
 {
 "WaterConsumption" : 1100.0,
 "DayOfWeek" : 5.0,
 "Prediction" : 1090.2142857142853
 },
 {
 "WaterConsumption" : 1300.0,
 "DayOfWeek" : 6.0,
 "Prediction" : 1114.821428571428
 },
 {
 "WaterConsumption" : 1400.0,
 "DayOfWeek" : 7.0,
 "Prediction" : 1139.4285714285704
 },
 {
 "WaterConsumption" : 850.0,
 "DayOfWeek" : 1.0,
 "Prediction" : 991.7857142857152
 },
 {
 "WaterConsumption" : 950.0,
 "DayOfWeek" : 2.0,
 "Prediction" : 1016.3928571428578
 },
 {
 "WaterConsumption" : 1100.0,
 "DayOfWeek" : 3.0,
 "Prediction" : 1041.0000000000002
 },
 {
 "WaterConsumption" : 1000.0,
 "DayOfWeek" : 4.0,
 "Prediction" : 1065.607142857143
 },
 {
 "WaterConsumption" : 1050.0,
 "DayOfWeek" : 5.0,
 "Prediction" : 1090.2142857142853
 },
 {
 "WaterConsumption" : 850.0,
 "DayOfWeek" : 6.0,
 "Prediction" : 1114.821428571428
 },
 {
 "WaterConsumption" : 950.0,
 "DayOfWeek" : 7.0,
 "Prediction" 

In [10]:
import spark.implicits._
import org.apache.spark.sql._
val lines2 = spark.readStream.format("org.apache.bahir.sql.streaming.mqtt.MQTTStreamSourceProvider").
                option("topic","iot-2/type/+/id/+/evt/+/fmt/+").
                option("clientId","a:drnkqk:subId3").
                option("username","a-drnkqk-7yxweb6ere").
                option("password","VOpK+Zl0p3_wlv4Ka?").
   	            option("cleanSession","true").
                load("tcp://drnkqk.messaging.internetofthings.ibmcloud.com:1883").as[(String,Timestamp)]

In [11]:
import org.apache.spark.sql.types._
import org.apache.spark.sql.functions._
import spark.implicits._

case class IotEventClass(devId: String, devType: String, category: String, waterLevel: String, lightIndicator:String)

val eventSchema = Encoders.product[IotEventClass].schema
val lines3 = lines2.select($"value" cast "string" as "json")
val events = lines2.select($"value" cast "string" as "json").
      select(from_json($"json", eventSchema) as "data").
      select("data.*").as[IotEventClass]

val query = events.writeStream.
     outputMode("append").
     option("truncate", "false").
     format("console").
     start()

-------------------------------------------
Batch: 0
-------------------------------------------
+------+-----------------+--------+----------+--------------+
|devId |devType          |category|waterLevel|lightIndicator|
+------+-----------------+--------+----------+--------------+
|ws1   |waterSump        |Water   |82        |null          |
|ws1   |waterSump        |Water   |83        |null          |
|ws1   |waterSump        |Water   |84        |null          |
|Light1|LightSensorDevice|Light   |null      |1             |
|Light1|LightSensorDevice|Light   |null      |2             |
|Light1|LightSensorDevice|Light   |null      |2             |
+------+-----------------+--------+----------+--------------+

-------------------------------------------
Batch: 1
-------------------------------------------
+------+-----------------+--------+----------+--------------+
|devId |devType          |category|waterLevel|lightIndicator|
+------+-----------------+--------+----------+--------------+

-------------------------------------------
Batch: 12
-------------------------------------------
+------+-----------------+--------+----------+--------------+
|devId |devType          |category|waterLevel|lightIndicator|
+------+-----------------+--------+----------+--------------+
|ws1   |waterSump        |Water   |1         |null          |
|ws1   |waterSump        |Water   |2         |null          |
|ws1   |waterSump        |Water   |3         |null          |
|Light1|LightSensorDevice|Light   |null      |1             |
|Light1|LightSensorDevice|Light   |null      |2             |
|Light1|LightSensorDevice|Light   |null      |2             |
+------+-----------------+--------+----------+--------------+

-------------------------------------------
Batch: 13
-------------------------------------------
+------+-----------------+--------+----------+--------------+
|devId |devType          |category|waterLevel|lightIndicator|
+------+-----------------+--------+----------+-------------

-------------------------------------------
Batch: 24
-------------------------------------------
+------+-----------------+--------+----------+--------------+
|devId |devType          |category|waterLevel|lightIndicator|
+------+-----------------+--------+----------+--------------+
|ws1   |waterSump        |Water   |100       |null          |
|ws1   |waterSump        |Water   |101       |null          |
|ws1   |waterSump        |Water   |102       |null          |
|Light1|LightSensorDevice|Light   |null      |1             |
|Light1|LightSensorDevice|Light   |null      |2             |
|Light1|LightSensorDevice|Light   |null      |2             |
+------+-----------------+--------+----------+--------------+

-------------------------------------------
Batch: 25
-------------------------------------------
+-----+---------+--------+----------+--------------+
|devId|devType  |category|waterLevel|lightIndicator|
+-----+---------+--------+----------+--------------+
|ws1  |waterSump|Water  

-------------------------------------------
Batch: 38
-------------------------------------------
+------+-----------------+--------+----------+--------------+
|devId |devType          |category|waterLevel|lightIndicator|
+------+-----------------+--------+----------+--------------+
|ws1   |waterSump        |Water   |64        |null          |
|ws1   |waterSump        |Water   |65        |null          |
|ws1   |waterSump        |Water   |66        |null          |
|Light1|LightSensorDevice|Light   |null      |1             |
|Light1|LightSensorDevice|Light   |null      |2             |
|Light1|LightSensorDevice|Light   |null      |2             |
+------+-----------------+--------+----------+--------------+

-------------------------------------------
Batch: 39
-------------------------------------------
+------+-----------------+--------+----------+--------------+
|devId |devType          |category|waterLevel|lightIndicator|
+------+-----------------+--------+----------+-------------

-------------------------------------------
Batch: 51
-------------------------------------------
+------+-----------------+--------+----------+--------------+
|devId |devType          |category|waterLevel|lightIndicator|
+------+-----------------+--------+----------+--------------+
|ws1   |waterSump        |Water   |163       |null          |
|ws1   |waterSump        |Water   |164       |null          |
|ws1   |waterSump        |Water   |165       |null          |
|Light1|LightSensorDevice|Light   |null      |1             |
|Light1|LightSensorDevice|Light   |null      |2             |
|Light1|LightSensorDevice|Light   |null      |2             |
+------+-----------------+--------+----------+--------------+

-------------------------------------------
Batch: 52
-------------------------------------------
+------+-----------------+--------+----------+--------------+
|devId |devType          |category|waterLevel|lightIndicator|
+------+-----------------+--------+----------+-------------

-------------------------------------------
Batch: 63
-------------------------------------------
+------+-----------------+--------+----------+--------------+
|devId |devType          |category|waterLevel|lightIndicator|
+------+-----------------+--------+----------+--------------+
|ws1   |waterSump        |Water   |262       |null          |
|ws1   |waterSump        |Water   |263       |null          |
|ws1   |waterSump        |Water   |264       |null          |
|Light1|LightSensorDevice|Light   |null      |1             |
|Light1|LightSensorDevice|Light   |null      |2             |
|Light1|LightSensorDevice|Light   |null      |2             |
+------+-----------------+--------+----------+--------------+

-------------------------------------------
Batch: 64
-------------------------------------------
+------+-----------------+--------+----------+--------------+
|devId |devType          |category|waterLevel|lightIndicator|
+------+-----------------+--------+----------+-------------

-------------------------------------------
Batch: 75
-------------------------------------------
+------+-----------------+--------+----------+--------------+
|devId |devType          |category|waterLevel|lightIndicator|
+------+-----------------+--------+----------+--------------+
|ws1   |waterSump        |Water   |361       |null          |
|ws1   |waterSump        |Water   |362       |null          |
|ws1   |waterSump        |Water   |363       |null          |
|Light1|LightSensorDevice|Light   |null      |1             |
|Light1|LightSensorDevice|Light   |null      |2             |
|Light1|LightSensorDevice|Light   |null      |2             |
+------+-----------------+--------+----------+--------------+

-------------------------------------------
Batch: 76
-------------------------------------------
+------+-----------------+--------+----------+--------------+
|devId |devType          |category|waterLevel|lightIndicator|
+------+-----------------+--------+----------+-------------

-------------------------------------------
Batch: 89
-------------------------------------------
+------+-----------------+--------+----------+--------------+
|devId |devType          |category|waterLevel|lightIndicator|
+------+-----------------+--------+----------+--------------+
|ws1   |waterSump        |Water   |460       |null          |
|ws1   |waterSump        |Water   |461       |null          |
|ws1   |waterSump        |Water   |462       |null          |
|Light1|LightSensorDevice|Light   |null      |1             |
|Light1|LightSensorDevice|Light   |null      |2             |
|Light1|LightSensorDevice|Light   |null      |2             |
+------+-----------------+--------+----------+--------------+

-------------------------------------------
Batch: 90
-------------------------------------------
+------+-----------------+--------+----------+--------------+
|devId |devType          |category|waterLevel|lightIndicator|
+------+-----------------+--------+----------+-------------

-------------------------------------------
Batch: 101
-------------------------------------------
+------+-----------------+--------+----------+--------------+
|devId |devType          |category|waterLevel|lightIndicator|
+------+-----------------+--------+----------+--------------+
|ws1   |waterSump        |Water   |559       |null          |
|ws1   |waterSump        |Water   |560       |null          |
|ws1   |waterSump        |Water   |561       |null          |
|Light1|LightSensorDevice|Light   |null      |1             |
|Light1|LightSensorDevice|Light   |null      |2             |
|Light1|LightSensorDevice|Light   |null      |2             |
+------+-----------------+--------+----------+--------------+

-------------------------------------------
Batch: 102
-------------------------------------------
+------+-----------------+--------+----------+--------------+
|devId |devType          |category|waterLevel|lightIndicator|
+------+-----------------+--------+----------+-----------

-------------------------------------------
Batch: 113
-------------------------------------------
+------+-----------------+--------+----------+--------------+
|devId |devType          |category|waterLevel|lightIndicator|
+------+-----------------+--------+----------+--------------+
|ws1   |waterSump        |Water   |55        |null          |
|ws1   |waterSump        |Water   |56        |null          |
|ws1   |waterSump        |Water   |57        |null          |
|Light1|LightSensorDevice|Light   |null      |1             |
|Light1|LightSensorDevice|Light   |null      |2             |
|Light1|LightSensorDevice|Light   |null      |2             |
+------+-----------------+--------+----------+--------------+

-------------------------------------------
Batch: 114
-------------------------------------------
+------+-----------------+--------+----------+--------------+
|devId |devType          |category|waterLevel|lightIndicator|
+------+-----------------+--------+----------+-----------